# Deep Learning - Exercise 1

## Covertype Data Set

Data set that was chosen for this exercise is the Covertype Data Set and the problem is a multiclass classification problem - predicting forest cover type from cartographic variables only (no remotely sensed data).

[Link to dataset in UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Covertype)

### Imports

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

### Read the data

In [2]:
df = pd.read_csv('covtype.data', names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                                          'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                                          'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 
                                          'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 
                                          'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 
                                          'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 
                                          'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 
                                          'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 
                                          'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
                                          'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
                                          'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 
                                          'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 
                                          'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 
                                          'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 
                                          'Soil_Type40', 'Cover_Type'])
label_column = 'Cover_Type'

X = df.loc[:, df.columns != label_column].astype(float)
y = df.loc[:, label_column]

### Prepare the data as input for the network

In [3]:
# Normalize the data
X_values = X.iloc[:, :10].values.reshape(-1, 10) 
min_max_scaler = MinMaxScaler()
X.iloc[:, :10] = min_max_scaler.fit_transform(X_values)
mlb = MultiLabelBinarizer()
y = pd.DataFrame(mlb.fit_transform(y.values.reshape(-1, 1)).astype(float))

### Split the data into train, test and validation

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Hyperparameter tuning

In [5]:
epochs = 10
learning_rate = 0.01
keep_probability = 0.75
max_norm_constraint = 3
regularization = 0.1
hidden_layer_size = 256
batch_size = 1000

### Build the network

In [6]:
# Reset
tf.reset_default_graph()

# Placeholders
x = tf.placeholder(tf.float32, [None, X_train.shape[1]], 'x')
y = tf.placeholder(tf.float32, [None, y_train.shape[1]], 'y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Regularization
regularizer = tf.contrib.layers.l2_regularizer(regularization)

#TODO - max norm constraint?

# Model
model = tf.contrib.layers.fully_connected(x, hidden_layer_size, tf.nn.relu, weights_regularizer=regularizer)
model = tf.nn.dropout(model, keep_prob)
model = tf.contrib.layers.fully_connected(model, y_train.shape[1], weights_regularizer=regularizer)
logits = tf.nn.softmax(model)

# Loss and Optimizer
cost = tf.reduce_mean(tf.squared_difference(logits, y))
cost_summary = tf.summary.scalar('cost', cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

summary_op = tf.summary.merge_all()

### Training and testing the network

In [7]:
def get_batch(X, y, current_batch_index):
    batch_start_index = current_batch_index * batch_size
    batch_end_index = (current_batch_index + 1) * batch_size
    batch_features = X.iloc[batch_start_index:batch_end_index].values
    batch_labels = y.iloc[batch_start_index:batch_end_index].values
    return batch_features, batch_labels

def print_stats(session, epoch, feature_batch, label_batch, cost, accuracy):
    feed_cost = {x: feature_batch, y: label_batch, keep_prob: 1.0 }
    feed_valid = {x: X_validate, y: y_validate, keep_prob: 1.0 }
    cost = session.run(cost, feed_cost)
    accuracy = session.run(accuracy, feed_valid)
    print 'Epoch {:>2}: '.format(epoch + 1) + 'cost: %.2f' % cost, 'accuracy: %.2f' % accuracy
    
def clear_model(model):
    !rm -rf model

In [8]:
save_model_path = 'model'
clear_model(save_model_path)
saver = tf.train.Saver()
train_summary_writer = tf.summary.FileWriter('train')
test_summary_writer = tf.summary.FileWriter('test')
print 'Training...'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        n_batches = X_train.shape[0] / batch_size
        for batch_i in range(n_batches):
            batch_features, batch_labels = get_batch(X_train, y_train, batch_i)
            _, summary = sess.run([optimizer, summary_op], feed_dict={x: batch_features, y: batch_labels, keep_prob: keep_probability})
            train_summary_writer.add_summary(summary, epoch * n_batches + batch_i)
        print_stats(sess, epoch, batch_features, batch_labels, cost, accuracy)
        test_batch_acc_total = 0
        test_batch_count = 0
        n_batches = X_test.shape[0] / batch_size
        for batch_i in range(n_batches):
            batch_features, batch_labels = get_batch(X_test, y_test, batch_i)
            acc, summary = sess.run([accuracy, summary_op], feed_dict={x:batch_features,y:batch_labels,keep_prob:1.0})
            test_batch_acc_total += acc
            test_batch_count += 1
            test_summary_writer.add_summary(summary, epoch * n_batches + batch_i)
        print 'Testing accuracy: %.2f\n' % (test_batch_acc_total / test_batch_count)
        saver.save(sess, save_model_path + '/model', global_step=epoch)
    saver.save(sess, save_model_path + '/model')

Training...
Epoch  1: cost: 0.11 accuracy: 0.46
Testing accuracy: 0.46

Epoch  2: cost: 0.10 accuracy: 0.48
Testing accuracy: 0.48

Epoch  3: cost: 0.10 accuracy: 0.49
Testing accuracy: 0.49

Epoch  4: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch  5: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch  6: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch  7: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch  8: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch  9: cost: 0.09 accuracy: 0.49
Testing accuracy: 0.49

Epoch 10: cost: 0.09 accuracy: 0.50
Testing accuracy: 0.50

